In [4]:
# List of SMILES compounds
smiles_compounds = [
    "O=C(Cc1cccc2ccccc12)Nc1n[nH]c2ccc(N3CCCS3(=O)=O)cc12",
    "COC(=O)NC[C@@H](NC(=O)c1ccc(-c2nc(C3CCOCC3)cnc2N)cc1F)c1cccc(Br)c1",
    "COc1ccccc1Nc1cc(Oc2cc(C)c(C)nc2-c2ccccn2)ccn1",
    "O=C(/C=C/CN1CCCC1)N1CCOc2cc3ncnc(Nc4ccc(F)c(Cl)c4)c3cc21",
    "O=C(Nc1cccc(Nc2cc3c(=O)[nH][nH]c(=O)c3cc2Cl)c1)c1cccc(Cl)c1",
    "Cc1cc(CNc2nc(Nc3cc(C4CC4)[nH]n3)cc(NC3CC4CCC(C3)N4C)n2)on1",
    "Cc1cc(-c2cc(O)ccc2Cl)cc2nnc(Nc3ccc(S(N)(=O)=O)cc3)nc12",
    "NS(=O)(=O)c1cccc(N/C=C2\C(=O)Nc3ccccc32)c1",
    "CC(=O)Nc1ccc2cnn(-c3cc(NC4CC4)n4ncc(C#N)c4n3)c2c1",
    "CS(=O)(=O)c1cccc(Nc2nccc(N(CC#N)c3c(Cl)ccc4c3OCO4)n2)c1",
    "Cc1cnc(-c2ccnc(C(C)(C)O)n2)cc1-n1c(C)cc(OCc2ccc(F)cc2F)c(Cl)c1=O",
    "Cc1ccc(C(=O)Nc2cc(C(F)(F)F)ccn2)cc1/C=C/n1cnc2cncnc21",
    "CNC(=O)c1cnn2ccc(N3C[C@@H](O)C[C@@H]3c3cccc(F)c3)nc12",
    "COc1cc2c(cc1OC1CCOC1)Cc1c-2n[nH]c1-c1ccc(C#N)cc1",
]

## K Means

In [5]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit import DataStructs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import numpy as np

# Generate ECFP descriptors
ecfps = [
    AllChem.GetMorganFingerprintAsBitVect(Chem.MolFromSmiles(smiles), 2)
    for smiles in smiles_compounds
]

# Calculate similarity matrix
similarity_matrix = []
for i in range(len(ecfps)):
    similarities = []
    for j in range(len(ecfps)):
        similarity = DataStructs.TanimotoSimilarity(ecfps[i], ecfps[j])
        similarities.append(similarity)
    similarity_matrix.append(similarities)

similarity_matrix = np.array(similarity_matrix)

# Cluster compounds using K-means
kmeans = KMeans(n_clusters=3)
kmeans.fit(similarity_matrix)

clusters = kmeans.labels_

# Evaluate clustering result
score = silhouette_score(similarity_matrix, clusters)

print("Silhouette score: ", score)
print("Cluster assignments: ", clusters)

Silhouette score:  0.03214939028636155
Cluster assignments:  [2 0 0 2 2 0 1 1 0 1 2 2 0 0]


/home/ilker/miniconda3/envs/thesis-work/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [6]:
np.set_printoptions(suppress=True)
np.set_printoptions(precision=3)

print(similarity_matrix)

[[1.    0.122 0.093 0.15  0.149 0.127 0.091 0.115 0.13  0.142 0.111 0.105
  0.141 0.107]
 [0.122 1.    0.128 0.148 0.104 0.128 0.116 0.096 0.13  0.14  0.142 0.118
  0.193 0.159]
 [0.093 0.128 1.    0.083 0.106 0.112 0.121 0.083 0.092 0.127 0.163 0.158
  0.115 0.149]
 [0.15  0.148 0.083 1.    0.181 0.086 0.167 0.091 0.117 0.168 0.159 0.155
  0.127 0.078]
 [0.149 0.104 0.106 0.181 1.    0.062 0.143 0.164 0.122 0.172 0.091 0.169
  0.122 0.052]
 [0.127 0.128 0.112 0.086 0.062 1.    0.099 0.053 0.126 0.107 0.088 0.092
  0.105 0.136]
 [0.091 0.116 0.121 0.167 0.143 0.099 1.    0.208 0.124 0.206 0.137 0.143
  0.101 0.078]
 [0.115 0.096 0.083 0.091 0.164 0.053 0.208 1.    0.101 0.178 0.061 0.098
  0.101 0.076]
 [0.13  0.13  0.092 0.117 0.122 0.126 0.124 0.101 1.    0.14  0.1   0.147
  0.2   0.139]
 [0.142 0.14  0.127 0.168 0.172 0.107 0.206 0.178 0.14  1.    0.132 0.116
  0.109 0.098]
 [0.111 0.142 0.163 0.159 0.091 0.088 0.137 0.061 0.1   0.132 1.    0.159
  0.142 0.07 ]
 [0.105 0.118 0.158 0

## Butina clustering

In [14]:
# TODO: Find out correct way to use Butina clustering
from rdkit.ML.Cluster import Butina

mols = [Chem.MolFromSmiles(smiles) for smiles in smiles_compounds]
fps = [AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=1024) for mol in mols]

distances = []
for i in range(len(fps)):
    sims = DataStructs.BulkTanimotoSimilarity(fps[i], fps[:i])
    distances.extend([1 - x for x in sims])

threshold = 0.2
clusters = Butina.ClusterData(distances, len(mols), threshold, isDistData=True)

# Convert the clusters into labels for the silhouette score
labels = np.zeros(len(mols))
for i, cluster in enumerate(clusters):
    for mol in cluster:
        labels[mol] = i

# Compute the silhouette score
silhouette_score = silhouette_score(np.array(list(fps)), labels, metric="jaccard")
print("Silhouette Score: ", silhouette_score)

array([13., 12., 11., 10.,  9.,  8.,  7.,  6.,  5.,  4.,  3.,  2.,  1.,
        0.])

In [ ]:
# FIXME: Not working
def quality_partition_index(clusters, distances):
    intra_cluster_distances = [
        min([distances[i][j] for j in cluster]) for i, cluster in enumerate(clusters)
    ]
    min_intra_cluster_distance = min(intra_cluster_distances)
    max_inter_cluster_distance = max(
        [distances[i][j] for i in range(len(distances)) for j in range(i)]
    )

    return min_intra_cluster_distance / max_inter_cluster_distance


qpi = quality_partition_index(clusters, distances)
print("Quality Partition Index: ", qpi)